In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [21]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
trainY = train_data.iloc[:,1]
trainX = pd.get_dummies(train_data[["Pclass", "Sex", "Age", "SibSp", "Parch"]])
age_mean = np.mean(trainX['Age'])
trainX['Age'] = trainX['Age'].fillna(age_mean)
trainX

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male
0,3,22.000000,1,0,False,True
1,1,38.000000,1,0,True,False
2,3,26.000000,0,0,True,False
3,1,35.000000,1,0,True,False
4,3,35.000000,0,0,False,True
...,...,...,...,...,...,...
886,2,27.000000,0,0,False,True
887,1,19.000000,0,0,True,False
888,3,29.699118,1,2,True,False
889,1,26.000000,0,0,False,True


In [23]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
testX = pd.get_dummies(test_data[["Pclass", "Sex", "Age", "SibSp", "Parch"]])
testX['Age'] = testX['Age'].fillna(age_mean)
testX

,Pclass,Age,SibSp,Parch,Sex_female,Sex_male
0,3,34.500000,0,0,False,True
1,3,47.000000,1,0,True,False
2,2,62.000000,0,0,False,True
3,3,27.000000,0,0,False,True
4,3,22.000000,1,1,True,False
...,...,...,...,...,...,...
413,3,29.699118,0,0,False,True
414,1,39.000000,0,0,True,False
415,3,38.500000,0,0,False,True
416,3,29.699118,0,0,False,True


Applying a GridSearchCV() on all paramters is time-comsuming. So I greedily search for the best parameter step by step.(Coordinate Descent)

In [24]:
rf_param_grid1 = {'n_estimators':[30,45,50,55]}
models = GridSearchCV(RandomForestClassifier(random_state=1),rf_param_grid1,scoring="accuracy").fit(trainX, trainY)
model1 = models.best_estimator_
print(models.best_params_)

{'n_estimators': 45}


In [25]:
rf_param_grid2 = {'max_depth':[3,5,10,15,20]}
models = GridSearchCV(model1,rf_param_grid2,scoring="accuracy").fit(trainX, trainY)
model2 = models.best_estimator_
print(models.best_params_)

{'max_depth': 5}


In [26]:
rf_param_grid3 = {'min_samples_split':range(2,5)}
models = GridSearchCV(model2,rf_param_grid3,scoring="accuracy").fit(trainX, trainY)
model3 = models.best_estimator_
print(models.best_params_)

{'min_samples_split': 4}


In [27]:
rf_param_grid4 = {'max_features':range(1,7)}
models = GridSearchCV(model3,rf_param_grid4,scoring="accuracy").fit(trainX, trainY)
model4 = models.best_estimator_
print(models.best_params_)

print("Training score: ")
print(accuracy_score(trainY, model4.predict(trainX)))

{'max_features': 1}
Training score: 
0.8372615039281706


In [29]:
predictions = model4.predict(testX)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Finish!")

Finish!


In [31]:
print(model4)

RandomForestClassifier(max_depth=5, max_features=1, min_samples_split=4,
                       n_estimators=45, random_state=1)
